In [ ]:
pip install langchain langchain-community langchain-google-genai faiss-cpu pypdf streamlit pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 102.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.8/475.8 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 103.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found exi

In [ ]:
from pyngrok import ngrok
# Get your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken
NGROK_AUTH_TOKEN = "36b3aogkVI94W0EneDY5gf9fecE_2hzPMZTmH54ooQqtGR3T"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
!pkill ngrok

In [ ]:
pip install docx2txt

In [ ]:
%%writefile my_app.py
import streamlit as st
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader # Make sure Docx2txtLoader is imported
# ... other imports (st, RecursiveCharacterTextSplitter, etc.)
import os

# ------------------- PAGE CONFIG -------------------
st.set_page_config(page_title="UrPdfFriend", layout="wide")


# ------------------- CUSTOM UI CSS -------------------
st.markdown("""
<style>

html, body, .stApp {
    background: linear-gradient(135deg, #4A148C, #880E4F, #FF00FF, #00FFFF); /* CHANGED COLORS */
    background-size: 400% 400%;
    animation: gradientBG 12s ease infinite;
    color: white !important;
}

@keyframes gradientBG {
    0% { background-position: 0% 50%; }
    50% { background-position: 100% 50%; }
    100% { background-position: 0% 50%; }
}

.block-container {
    background: transparent !important;
    padding-top: 1rem;
}

/* Header Image */
.header-container {
    display: flex;
    justify-content: center;
    margin-bottom: 10px;
}

.header-img {
    width: 60%;
    border-radius: 16px;
    box-shadow: 0 0 25px rgba(0,0,0,0.45);
}

/* Chat bubbles */
.chat-message {
    border-radius: 12px;
    padding: 14px 18px;
    margin-bottom: 1rem;
    max-width: 80%;
    backdrop-filter: blur(10px);
    color: #fff;
}

.user-msg {
    background: rgba(0, 180, 255, 0.35);
    margin-left: auto;
}

.bot-msg {
    background: rgba(255, 255, 255, 0.15);
    border: 1px solid rgba(255,255,255,0.25);
}

/* File Uploader */
.stFileUploader {
    background: rgba(255, 255, 255, 0.08);
    padding: 14px;
    border-radius: 12px;
    backdrop-filter: blur(6px);
}

.stFileUploader label div {
    color: #ffffff !important;
    font-size: 18px !important;
}

/* Chat input */
.stChatInput input {
    background-color: rgba(0,0,0,0.35) !important;
    color: white !important;
    border-radius: 10px !important;
    border: 1px solid rgba(255,255,255,0.25) !important;
}

/* Remove Streamlit white header */
div[data-testid="stHeader"] {
    background: transparent !important;
}

/* Remove top toolbar */
div[data-testid="stToolbar"] {
    display: none !important;
}

/* Fix Browse Files button style */
.stFileUploader button[kind="secondary"] {
    background-color: rgba(0, 0, 0, 0.55) !important;
    color: #ffffff !important;
    border-radius: 10px !important;
    padding: 8px 16px !important;
    border: 1px solid rgba(255,255,255,0.25) !important;
}

/* Hover effect */
.stFileUploader button[kind="secondary"]:hover {
    background-color: rgba(0, 0, 0, 0.75) !important;
    color: #ffffff !important;
    border: 1px solid rgba(255,255,255,0.45) !important;
    cursor: pointer !important;
}
</style>
""", unsafe_allow_html=True)


# ------------------- HEADER IMAGE -------------------
st.markdown(
    """
    <div class='header-container'>
        <img src="https://tse3.mm.bing.net/th/id/OIP.egCUtrB2A1lXPg9qhwCxlAHaHa?w=187&h=186&c=7&r=0&o=7&dpr=1.5&pid=1.7&rm=3" class="header-img">
    </div>
    """,
    unsafe_allow_html=True
)

st.markdown("<h1 style='text-align:center; color:white;'>📚 Reading is hard? Lemme Help you!</h1>", unsafe_allow_html=True)
st.markdown("<p style='text-align:center; color:#f0f0f0; font-size:18px;'>Ask anything from your uploaded PDF with accurate citations</p>", unsafe_allow_html=True)


# ------------------- MODIFIED RAG LOGIC FOR MULTIPLE FILES -------------------

# 1. Change file_uploader to accept multiple files (PDF and DOCX)
uploaded_files = st.file_uploader(
    "Upload PDF(s) or Word Document(s)",
    type=["pdf", "docx"], # Added 'docx'
    accept_multiple_files=True
)

if uploaded_files:
    # 2. Process all uploaded files
    all_pages = []

    # Use st.spinner for a better UX while processing files
    with st.spinner(f"Processing {len(uploaded_files)} file(s)..."): # Updated count and text
        for uploaded_file in uploaded_files:
            # Save the file to a temporary location
            temp_file_name = uploaded_file.name # Generic file name variable
            file_extension = temp_file_name.split('.')[-1].lower() # Get file extension

            with open(temp_file_name, "wb") as f:
                f.write(uploaded_file.read())

            # Use the appropriate loader based on the file extension
            if file_extension == "pdf":
                loader = PyPDFLoader(temp_file_name)
            elif file_extension == "docx":
                loader = Docx2txtLoader(temp_file_name)
            else:
                # Should not happen due to 'type' filter, but good practice
                st.warning(f"Skipping unsupported file: {temp_file_name}")
                os.remove(temp_file_name)
                continue

            pages = loader.load()
            all_pages.extend(pages)

            # Clean up the temporary file
            os.remove(temp_file_name)

        # Ensure content was successfully extracted
        if not all_pages:
             st.error("No content could be extracted from the uploaded files.")
             st.stop() # Stop further execution if no documents are loaded


        # 3. Split documents from all pages combined
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=800,
            chunk_overlap=150
        )

        documents = splitter.split_documents(all_pages)

        # 4. Continue with RAG setup using the combined documents
        os.environ["GOOGLE_API_KEY"] = "Auth_token"

        model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)
        embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

        # Create a single Vector Store from all documents
        vectorstore = FAISS.from_documents(documents, embeddings)
        retriever = vectorstore.as_retriever(search_type="similarity", k=5)

        rag_prompt = PromptTemplate(
            input_variables=["context", "question"],
            template="""
You must answer the user's question ONLY using the text in the given context.
STRICT RULES:
- DO NOT paraphrase.
- Use ONLY exact sentences/phrases present in the context.
- If answer requires 1 mark, give very short exact lines.
- If answer requires 10 marks, give long exact lines.
- Add page number citations like: (Source: X)

CONTEXT:
{context}

QUESTION:
{question}

ANSWER (with source):
"""
        )

        # The format_docs function is updated to be robust for documents with or without page numbers (e.g., Word documents)
        def format_docs(docs):
            # This links to the original document source and page number (if available)
            return "\n\n".join([f"(Source: {d.metadata.get('source')} Page: {d.metadata.get('page', 'N/A')}):\n{d.page_content}" for d in docs])

        rag_chain = (
            RunnableParallel({
                "context": RunnableLambda(lambda x: retriever.invoke(x["question"])) | RunnableLambda(format_docs),
                "question": RunnableLambda(lambda x: x["question"])
            })
            | rag_prompt
            | model
            | StrOutputParser()
        )

        # Initialize history if it's the first run with files
        if "history" not in st.session_state:
            st.session_state.history = []

        # CHAT HISTORY UI (Moved inside the 'if uploaded_files' block)
        for msg in st.session_state.history:
            bubble_class = "user-msg" if msg["role"] == "user" else "bot-msg"
            st.markdown(
                f"<div class='chat-message {bubble_class}'>{msg['content']}</div>",
                unsafe_allow_html=True
            )

        user_prompt = st.chat_input("Ask anything from the files...") # Updated chat input text

        if user_prompt:
            st.session_state.history.append({"role": "user", "content": user_prompt})
            st.markdown(f"<div class='chat-message user-msg'>{user_prompt}</div>", unsafe_allow_html=True)

            with st.spinner("Thinking..."):
                response = rag_chain.invoke({"question": user_prompt})

            st.markdown(f"<div class='chat-message bot-msg'>{response}</div>", unsafe_allow_html=True)

            st.session_state.history.append({"role": "assistant", "content": response})

else:
    # 5. Handle the case where no files are uploaded
    st.info("Upload one or more PDF or Word documents to start chatting.") # Updated info message

Overwriting my_app.py


In [ ]:
from pyngrok import ngrok

# Terminate any existing ngrok tunnels started by pyngrok
ngrok.kill()

public_url = ngrok.connect(8501)
print("Public URL:", public_url)

!streamlit run my_app.py --server.port 8501 &> /dev/null &

Public URL: NgrokTunnel: "https://sewerlike-predecisively-beckett.ngrok-free.dev" -> "http://localhost:8501"
